In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import pandas as pd

# Carregar as planilhas
df_base = pd.read_csv('Base.csv')
df_culturas_pragas = pd.read_csv('Culturas_e_pragas.csv')

# Visualizar as primeiras linhas das planilhas
print(df_base.head())
print(df_culturas_pragas.head())

In [ ]:
import torch
from transformers import BertTokenizer, BertModel

# Carregar o modelo BERT e o tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def processar_respostas_com_bert(respostas):
    # Converte as respostas para uma string única
    texto = " ".join(respostas)
    # Tokenizar o texto
    inputs = tokenizer(texto, return_tensors="pt", padding=True, truncation=True)
    # Passa pelo modelo BERT
    with torch.no_grad():
        outputs = model(**inputs)
    # Extrai o embedding do CLS token para diagnóstico
    embeddings = outputs.last_hidden_state[:, 0, :].mean(dim=1)
    return embeddings

def diagnostico_e_tratamento(respostas):
    # Processa as respostas e obtém os embeddings
    embeddings = processar_respostas_com_bert(respostas)

    # Exemplo de lógica para diagnóstico simplificado
    # Aqui você poderia implementar uma lógica de similaridade para comparar com dados da planilha
    diagnostico = "Pragas do Solo"  # Este é um valor fixo para exemplo; você pode mudar

    # Busca o tratamento na planilha de acordo com o diagnóstico encontrado
    tratamento = df_culturas_pragas[df_culturas_pragas['Diagnostico'] == diagnostico].to_dict(orient='records')
    if not tratamento:
        return {"error": "Diagnóstico não encontrado"}

    return {
        "diagnostico": diagnostico,
        "tratamento_nivel_1": tratamento[0]["Tratamento Nível 1"],
        "tratamento_nivel_2": tratamento[0]["Tratamento Nível 2"],
        "tratamento_nivel_3": tratamento[0]["Tratamento Nível 3"]
    }

In [ ]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok

# Inicializar o aplicativo Flask
app = Flask(__name__)
run_with_ngrok(app)  # Inicia o ngrok para expor a API

@app.route('/diagnostico', methods=['POST'])
def diagnostico():
    respostas = request.json.get('respostas')

    if not respostas:
        return jsonify({"error": "Respostas não fornecidas"}), 400

    # Chama a função para diagnosticar e retornar o tratamento
    resultado = diagnostico_e_tratamento(respostas)

    return jsonify(resultado)

# Endpoint básico para verificar se a API está ativa
@app.route('/')
def home():
    return jsonify({"message": "API do Diagnóstico de Pragas está ativa!"})

# Iniciar a aplicação no Google Colab
app.run()

In [ ]:
# Pra chamar a API


curl -X POST https://<ngrok-id>.ngrok.io/diagnostico \
-H "Content-Type: application/json" \
-d '{"respostas": ["Milho", "Folhas amareladas", "Seca prolongada"]}'
